In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [8]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# Initialize MLflow
mlflow.set_experiment("deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/08/24 06:55:20 INFO mlflow.tracking.fluent: Experiment with name 'deployment model' does not exist. Creating a new experiment.
2025/08/24 06:55:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 06:55:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/716225462624211275/runs/fd0a8894394f46979ac219be2bc0ea49
🧪 View experiment at: http://localhost:5000/#/experiments/716225462624211275


2025/08/24 06:55:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 06:55:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/716225462624211275/runs/eeb98dd6bfe541e5a910d2d6ee86abaf
🧪 View experiment at: http://localhost:5000/#/experiments/716225462624211275


2025/08/24 06:55:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 06:55:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/716225462624211275/runs/27a7b6af00dc46ed87d4330035fc3dac
🧪 View experiment at: http://localhost:5000/#/experiments/716225462624211275


2025/08/24 06:55:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 06:55:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/716225462624211275/runs/0741c429778f4cb794ea1d44f9a54fa7
🧪 View experiment at: http://localhost:5000/#/experiments/716225462624211275


Register the Model

In [16]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/08/24 07:12:17 WARNING mlflow.tracking._model_registry.fluent: Run with id 0741c429778f4cb794ea1d44f9a54fa7 has no artifacts at artifact path 'model', registering model based on models:/m-f824ea3af46042feb0f15bab2749726b instead
2025/08/24 07:12:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 3
Created version '3' of model 'XGB-Smote'.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/716225462624211275/runs/0741c429778f4cb794ea1d44f9a54fa7
🧪 View experiment at: http://localhost:5000/#/experiments/716225462624211275


Load the Model

In [17]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Transition the Model to Production server

In [19]:
current_model_uri = f"models:/{model_name}@appserver"
production_model_name = "finalproduction"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'finalproduction'.
Copied version '3' of model 'XGB-Smote' to version '1' of model 'finalproduction'.


<ModelVersion: aliases=[], creation_timestamp=1756020095838, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1756020095838, metrics=None, model_id=None, name='finalproduction', params=None, run_id='0741c429778f4cb794ea1d44f9a54fa7', run_link='', source='models:/XGB-Smote/3', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [ ]:
'''
# ignore below code

model_version = 2
prod_model_uri = f"models:/{production_model_name}@productionserver2"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]
'''